Concatenated BLSTM(128) with CNN(2*64) with attention

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, LSTM,Bidirectional,  Concatenate, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,roc_curve,auc,roc_auc_score
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D,Flatten,Attention,Reshape
from sklearn.preprocessing import LabelEncoder,StandardScaler
from keras.layers import Input
from keras.callbacks import EarlyStopping
np.random.seed(0)
import random
random.seed(0)
import tensorflow as tf
tf.random.set_seed(0)

In [ ]:
data=pd.read_csv('/content/sample_data/satisfaction.csv')
data=data.dropna()
x=data.drop(columns=['satisfaction'])
y=data['satisfaction']
x=x.apply(LabelEncoder().fit_transform)
y=LabelEncoder().fit_transform(y)
x=np.array(x)
y=np.array(y)
scaler=StandardScaler()
x=scaler.fit_transform(x)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_train=x_train.reshape(x_train.shape[0],x_train.shape[1],1)
x_test=x_test.reshape(x_test.shape[0],x_test.shape[1],1)

In [ ]:
input_layer=Input(shape=(x_train.shape[1],1))
model=Sequential()
lstm_layer=(Bidirectional(LSTM(128,activation='relu',return_sequences=True)))(input_layer)
attention_out=Attention()([lstm_layer,lstm_layer])
attention_out=GlobalAveragePooling1D()(attention_out)
attention_out=Reshape((1,256))(attention_out)
attention_out= Dense(64, activation='relu')(attention_out)
cnn_out=Conv1D(filters=64,kernel_size=3,activation='relu')(input_layer)
cnn_out=MaxPooling1D(pool_size=2)(cnn_out)
cnn_out=Flatten()(cnn_out)
cnn_out=Dense(64,activation='relu')(cnn_out)
cnn_out=Dense(64,activation='relu')(cnn_out)
cnn_out=Reshape((1,64))(cnn_out)
combine_output=Concatenate()([attention_out,cnn_out])
output = Dense(1, activation='sigmoid')(Flatten()(combine_output))
model=Model(inputs=input_layer,outputs=output)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
blstm=model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_test,y_test))
loss=model.evaluate(x_test,y_test)
print('loss is=',loss)
y_pred=model.predict(x_test)
y_pred=np.round(y_pred)
print('accuracy_score is=',accuracy_score(y_test,y_pred))
print('recall_score is=',recall_score(y_test,y_pred))
print('precision_score is=',precision_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
fpr,tpr,threshold=roc_curve(y_test,y_pred)
roc_auc=auc(fpr,tpr)
plt.figure()
plt.plot(fpr,tpr,color='blue',label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
auc_score=roc_auc_score(y_test,y_pred)
print('AUC score is=',auc_score)
plt.plot(blstm.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()
w=recall_score(y_test,y_pred)
e=precision_score(y_test,y_pred)
F1=(w*e*2)/(w+e)
print('F1 is=',F1)